In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 10 10:34:21 2017

** ADVANCED LANE FINDING **
Project #3 of the first term of Udacity's Self Driving Car nanodegree program

Objective of this project is to detect lane boundaries and estimate lane 
curvature and vehicle position.
Note: This script works only on videos. A different script exists for processing images.

The steps of the pipeline are the following:
* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

@author: greliert
"""

#%% import useful modules
import numpy as np
import cv2
import pickle
from pipeline_functions import *
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
class Lane():
    def __init__(self,calib_dir):
        self.left_fit = np.array([[0,0,0]],dtype=np.float64)          # polynomial coefficients for left line
        self.right_fit = np.array([[0,0,0]],dtype=np.float64)         # polynomial coefficients for right line
        self.left_fit_smo = np.array([[0,0,0]],dtype=np.float64)          # smoothed polynomial coefficients for left line
        self.right_fit_smo = np.array([[0,0,0]],dtype=np.float64)         # smoothed polynomial coefficients for right line
        self.left_curverad = [0]     # left radius of curvature
        self.right_curverad = [0]   # right radius of curvature
        self.offset_car = [0]        # offset from center of lane  
        self.left_flag = [0]          # indicates if value is valid for left line
        self.right_flag = [0]         # indicates if value is valid for right line
        self.left_flag_smo = [0]      # indicates if smoothed value is valid for left line
        self.right_flag_smo = [0]     # indicates if smoothed value is valid for left line
        # load calibration matrix and distortion coefficients
        self.load_calibration(calib_dir)  
    
    def load_calibration(self,calib_dir):
        '''
        Load camera calibration matrix and distortion coefficients
        Input:
        - calib_dir: path of the directory containing the calibration parameters
        '''
        dist_pickle = pickle.load(open(calib_dir+'calib.p', "rb" ) )
        self.mtx = dist_pickle["mtx"]
        self.dist = dist_pickle["dist"]
  
    def process_image(self,img):
        '''
        Detect lane, compute radius of curvatures of l/r lines and car offset from
        the center of the line
        Inputs:
        - img: input image (RGB)
        Outputs:
        - out_img: processed image (RGB)
        '''
        # 1) Apply a distortion correction to raw images.
        img_undist = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        
        # 2) Use color transforms, gradients, etc., to create a thresholded binary image.
        ksize = 3   # kernel size for gradient computation
        # Compute masks based on gradients (x,y,magnitude,direction)
        gradx = abs_sobel_thresh(img_undist, 'x', ksize, (50, 255),'S')
        grady = abs_sobel_thresh(img_undist, 'y', ksize, (50, 255),'S')
        mag_binary = mag_thresh(img_undist, ksize, (50, 255),'S')
        dir_binary = dir_thresh(img_undist, ksize, (35/180.*np.pi, 75/180.*np.pi),'S')
        # Combine gradient masks
        grad_binary = np.zeros_like(dir_binary)
        sel = 1  # select mask
        if sel==1:
            grad_binary[((gradx == 1) & (grady == 1))] = 1
        elif sel==1:
            grad_binary[(mag_binary == 1) & (dir_binary == 1)] = 1 
        elif sel==2:
            grad_binary[(mag_binary == 1)] = 1 
        
        # Compute mask based on color components
        # Saturation
        satu_binary = color_thresh(img_undist, 'HLS', [[0,255],[0,255],[100,255]])  
        # Yellow: Hue value between 
        yel_binary = color_thresh(img_undist, 'HLS', [[18,30],[0,255],[80,255]])
        # White: Hue value between 
        whi_binary = color_thresh(img_undist, 'GRAY', [[200,255]])
        # Combine color and gradient mask
        mask = np.zeros_like(dir_binary)
        #mask[(satu_binary == 1) | (grad_binary == 1)] = 1
        mask[(grad_binary+whi_binary+yel_binary)>=1] = 1
        
        # 3) Apply a perspective transform to rectify binary image ("birds-eye view").
        y_size, x_size= mask.shape
        
        # Select the 4 quadrangle vertices in the original images that will be used
        # for the perspective transform
        xdl, ydl = 200, y_size        # down left
        xdr, ydr = x_size-200, y_size # down right
        xul, yul = 637, 420           # up left
        xur, yur = 643, 420           # up right
        al = float(ydl-yul)/float(xdl-xul)
        ar = float(ydr-yur)/float(xdr-xur)
        bl = ydl-al*xdl
        br = ydr-ar*xdr
        yul, yur = 450, 450 
        xul, xur = (yul-bl)/al, (yur-br)/ar 
        src_points = np.float32([[xdl,ydl],[xdr,ydr],[xur,yur],[xul,yul]])
        # Select the corresponding 4 points in the destination images to which will be mapped the
        # source points
        # we take same y-values. For x, we shrink the rectangle width (using parameter
        # off_x so as not to lose the sides of the image
        off_x = 100
        dst_points = np.float32([[xdl+off_x,ydl],[xdr-off_x,ydr],[xdr-off_x,0],[xdl+off_x,0]])
        
        # compupte lane width in the warp image
        lane_width = xdr-xdl-2*off_x
        
        # Warp image
        img_warp, M = rectify_image(img_undist, src_points, dst_points)
        mask_warp, M = rectify_image(mask, src_points, dst_points)
        
        # 4) Detect line pixels and fit to find the lane boundary.
        #  find_lines returns the 2nd order polynomial coefficients
        # if smoothed values are valid, use them as input values to find lines
        left_fit, right_fit = find_lines(mask_warp,self.left_fit[-1],self.right_fit[-1],self.left_flag_smo[-1],self.right_flag_smo[-1])  
        # check consistence of estimated values (check absolute values, and difference with previous value)
        left_flag, right_flag = sanity_check(left_fit, right_fit, self.left_fit_smo[-1], self.right_fit_smo[-1],
                                             self.left_flag_smo[-1],self.right_flag_smo[-1])
        
        # Compute smoothed coefficients
        n_win = 75  # size of the smoothing window
        n_min = 50  # min number of samples to apply smoothing
        n_last = 5  # number of trailing values that must be different from zero
        poly_order = 2  # polynomial order
        left_fit_smo, left_flag_smo = smooth(np.append(self.left_fit[1:,:],left_fit[None,:],0),
                                             np.append(self.left_flag[1:],left_flag),n_win,n_min,n_last,poly_order)
        right_fit_smo, right_flag_smo = smooth(np.append(self.right_fit[1:,:],right_fit[None,:],0),
                                               np.append(self.right_flag[1:],right_flag),n_win,n_min,n_last,poly_order)
        
        flag = 1  # flag indicating if the lines are valid for plotting
        
        # if smoothing invalid for one line, use values from other line
        if (left_flag_smo == 0) & (right_flag_smo==1):
            left_fit_cor = right_fit_smo-np.array([0,0,lane_width])
            left_fit_smo, left_flag_smo = smooth(np.append(self.left_fit[1:,:],left_fit_cor[None,:],0),
                                             np.append(self.left_flag[1:],right_flag_smo),n_win,n_min,n_last,poly_order)
        elif (left_flag_smo == 1) & (right_flag_smo==0):
            right_fit_cor = left_fit_smo+np.array([0,0,lane_width])
            right_fit_smo, right_flag_smo = smooth(np.append(self.right_fit[1:,:],right_fit_cor[None,:],0),
                                               np.append(self.right_flag[1:],left_flag_smo),n_win,n_min,n_last,poly_order)
        elif (left_flag_smo == 0) & (right_flag_smo==0):        
            if (left_flag == 0) & (right_flag==1):
                left_fit_cor = right_fit-np.array([0,0,lane_width])
                left_fit_smo, left_flag_smo = smooth(np.append(self.left_fit[1:,:],left_fit_cor[None,:],0),
                                                 np.append(self.left_flag[1:],right_flag),n_win,n_min,n_last,poly_order)
            elif (left_flag==1) & (right_flag == 0):
                right_fit_cor = left_fit+np.array([0,0,lane_width])
                right_fit_smo, right_flag_smo = smooth(np.append(self.right_fit[1:,:],right_fit_cor[None,:],0),
                                                   np.append(self.right_flag[1:],left_flag),n_win,n_min,n_last,poly_order)
            elif (left_flag==0) & (right_flag == 0):
                # last case: all flags to zero. Don't plot the lines
                flag = 0
                
        # 5) Determine the curvature of the lane and vehicle position with respect to center.
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720. # meters per pixel in y dimension
        xm_per_pix = 3.7/lane_width # meters per pixel in x dimension
        
        # convert polynomial coefficients from pixels to meters
        left_fit_cr = xm_per_pix*left_fit_smo/np.array([ym_per_pix**2,ym_per_pix,1])
        right_fit_cr = xm_per_pix*right_fit_smo/np.array([ym_per_pix**2,ym_per_pix,1])
        
        # Calculate the new radii of curvature at different along vertical axis
        y_eval = np.arange(0,y_size,10)*ym_per_pix
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
        # Compute offset from the center of the line
        ploty = np.arange(y_size)
        left_fitx = left_fit_smo[0]*ploty**2 + left_fit_smo[1]*ploty + left_fit_smo[2]
        right_fitx = right_fit_smo[0]*ploty**2 + right_fit_smo[1]*ploty + right_fit_smo[2]
        # take the last value for the base of the image
        offset_car = ((left_fitx[-1]+right_fitx[-1])/2-x_size/2)*xm_per_pix
        
        if flag == 1: # plot lane only if flag is valid
            # 6) Warp the detected lane boundaries back onto the original image.
            # Create an image to draw the lines on
            warp_zero = np.zeros_like(mask_warp).astype(np.uint8)
            color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
            # Recast the x and y points into usable format for cv2.fillPoly()
            pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
            pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
            pts = np.hstack((pts_left, pts_right))
            # Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
            # Draw the lane onto the warped blank image
            cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

            # Warp the blank back to original image space using inverse perspective matrix (Minv)
            Minv = np.linalg.inv(M)
            newwarp = cv2.warpPerspective(color_warp, Minv, (x_size, y_size)) 
            # Combine the result with the original image
            out_img = cv2.addWeighted(img_undist, 1, newwarp, 0.3, 0)
            # print curvature radius on image
            curverad = np.mean((left_curverad + right_curverad)/2)
            #print('Rad={}'.format(curverad))
            cv2.putText(out_img,'Curv. Rad. = {:.1f} m'.format(curverad), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, [255,255,255])
            cv2.putText(out_img,'Offset = {:.2f} m'.format(offset_car), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, [255,255,255])
        else:
            out_img = img_undist
            
        # Appending the outputs to lane object
        self.left_fit = np.append(self.left_fit,left_fit[None,:],0)
        self.right_fit = np.append(self.right_fit,right_fit[None,:],0)
        self.left_flag = np.append(self.left_flag,left_flag)
        self.right_flag = np.append(self.right_flag,right_flag)
        self.left_fit_smo = np.append(self.left_fit_smo,left_fit_smo[None,:],0)
        self.right_fit_smo = np.append(self.right_fit_smo,right_fit_smo[None,:],0)
        self.left_curverad = np.append(self.left_curverad,left_curverad)
        self.right_curverad = np.append(self.right_curverad,right_curverad)
        self.offset_car = np.append(self.offset_car,offset_car)
        self.left_flag_smo = np.append(self.left_flag_smo,left_flag_smo)
        self.right_flag_smo = np.append(self.right_flag_smo,right_flag_smo)
        
        # Save lane object
        if (len(self.offset_car)%50)==0:
            pickle.dump(self, open('lane2.p', "wb" ) )

        return out_img

In [3]:
#%% VIDEO PROCESSING
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Parameters
calib_dir = './camera_cal/'           # Camera calibration directory
video_path = "./test_images_videos/project_video.mp4"      # Video file path

# Video output path
video_output_path = video_path[:-4]+'_output.mp4'
# create a line object
lane = Lane(calib_dir)   
# Load video
clip1 = VideoFileClip(video_path)
#clip1 = VideoFileClip(video_path).subclip(0,50)

In [4]:
# Process video
out_clip = clip1.fl_image(lane.process_image)
# Write output video
out_clip.write_videofile(video_output_path, audio=False)

[MoviePy] >>>> Building video ./test_images_videos/project_video_output.mp4
[MoviePy] Writing video ./test_images_videos/project_video_output.mp4


100%|█████████████████████████████████████▉| 1260/1261 [08:33<00:00,  2.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./test_images_videos/project_video_output.mp4 



In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output_path))